# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。
回忆一下在 :numref:`sec_model_construction`对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。


In [18]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

让我们向该层提供一些数据，验证它是否能按预期工作。


In [19]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以[**将层作为组件合并到更复杂的模型中**]。


In [20]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [21]:
Y = net(torch.rand(4, 8))
print(Y)
Y.mean()

tensor([[-0.1315,  0.2825, -0.0141,  0.1366, -0.3708,  0.4238, -0.0427, -0.5763,
          0.1321,  0.0577,  0.6263, -0.5594, -0.6086, -0.3592,  0.9616,  0.6024,
          0.4735, -0.2972,  0.9424, -0.1184,  0.0457, -0.3545,  0.3688, -1.1623,
         -0.7682,  0.3067,  0.6842,  0.6159, -0.1157, -0.0697,  0.3461, -0.0831,
         -0.1799, -0.2460, -0.4183,  0.6505, -0.3914, -0.4517, -0.3404, -0.1135,
          0.5236,  0.2700,  0.4880, -0.1711,  0.6260, -0.6326, -0.1620,  0.7112,
         -1.0369, -0.7556,  0.0082,  0.3500,  0.2307,  0.1353,  0.9889, -0.5606,
         -0.3582, -0.1032, -0.4990,  1.0232,  0.4340,  0.0137,  0.8552,  0.9319,
          0.0800,  0.4218,  0.0543, -0.1338, -0.5242, -1.0258, -0.5044, -0.7414,
         -0.6582,  0.4325, -0.4005, -0.4117, -0.4024,  0.1501, -0.2236,  0.1007,
          0.6182, -0.0742, -0.0316,  0.0490,  0.0956,  0.3021,  0.1952, -0.7597,
          0.3634,  0.8629,  0.0963, -0.0934, -0.3613,  0.6510, -0.6861,  0.2040,
         -0.3028, -0.3938,  

tensor(-5.5879e-09, grad_fn=<MeanBackward0>)

## [**带参数的层**]

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。


In [22]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化`MyLinear`类并访问其模型参数。


In [23]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.1050,  0.2539, -0.6508],
        [ 1.0387, -0.3283,  1.7847],
        [-0.7417, -0.5461,  1.2678],
        [ 1.5695,  1.0861, -2.3013],
        [-1.7376,  0.4236, -0.8101]], requires_grad=True)

我们可以[**使用自定义层直接执行前向传播计算**]。


In [38]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0868, 0.0341],
        [0.0000, 0.0000, 0.7865]])

我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。


In [39]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个返回输入数据的傅立叶系数前半部分的层。


[Discussions](https://discuss.d2l.ai/t/1835)
